# LIBRARY

In [1]:
from __future__ import division
from nltk.tokenize import word_tokenize, RegexpTokenizer
from collections import defaultdict
import pandas as pd
import requests
import sqlite3
from sqlite3 import Error
import os
import re
import sys, time
import nltk
import numpy as np
#stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Connecting to Database

In [23]:
#connecting to database
conn = sqlite3.connect("DBNews.db")
dataset = pd.read_sql_query("SELECT * FROM t_comments", conn)
cursor = conn.cursor()

In [24]:
pd.set_option('display.max_colwidth', -1)

In [25]:
dataset

,comment_id,comment,baseline,correct_comment,comment_ptr
0,4207096,"WKWKWK, GAK SEKALIAN PAK, SELAWAT KE DPR... KAN BANYAK KORUPTOR DISANA",-1,None,1
1,4207096,Mantab.. ini br dakwah yg sejati.. kl d tempat umum mah dh biasa . Ini luar biasa.�,1,None,2
2,4207096,Salut buat Gus Miftah,1,None,3
3,4207096,yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo \/ sunan kali jaga...smua ada cara demi kebaikan... asal jng ajk dg cara berjinah .... lakukan dg porsi dan sikon yg tepat,1,None,4
4,4207096,Ada adabnya sholawat. Lebih baik ajak ke majelis ilmu..,1,None,5
5,4207096,"Setiap ulama punya jalan dan cara dakwah masing2.., jika kamu melihat kemunkaran lawan dengan tanganmu(kekuatanmu) jika tidak mampu lawan dengan mulutmu jika tidak mampu juga dengan hatimu itulah selemah lemahnya iman.., itu sudah menjadi kewajiban para ulama, kalo yg komen masih ada yg nyinyirin ulama berarti otaknya sudah disisipin politik..",1,None,6
6,4207096,Umpanin nocannya yak...,0,None,7
7,4207096,Itulah ceramah yg sebenarnya.,0,None,8
8,4207096,Besok bugil aja selawat,-1,None,9
9,4207096,"Bedanya Gus Miftah dengan FPI / HTI bagaikan surga dan neraka. Di surga penuh kasih, di neraka penuh penghancuran.",0,None,10


In [26]:
#ambil yg kolom comment aja
dataset = dataset[['comment']]

In [27]:
dataset

,comment
0,"WKWKWK, GAK SEKALIAN PAK, SELAWAT KE DPR... KAN BANYAK KORUPTOR DISANA"
1,Mantab.. ini br dakwah yg sejati.. kl d tempat umum mah dh biasa . Ini luar biasa.�
2,Salut buat Gus Miftah
3,yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo \/ sunan kali jaga...smua ada cara demi kebaikan... asal jng ajk dg cara berjinah .... lakukan dg porsi dan sikon yg tepat
4,Ada adabnya sholawat. Lebih baik ajak ke majelis ilmu..
5,"Setiap ulama punya jalan dan cara dakwah masing2.., jika kamu melihat kemunkaran lawan dengan tanganmu(kekuatanmu) jika tidak mampu lawan dengan mulutmu jika tidak mampu juga dengan hatimu itulah selemah lemahnya iman.., itu sudah menjadi kewajiban para ulama, kalo yg komen masih ada yg nyinyirin ulama berarti otaknya sudah disisipin politik.."
6,Umpanin nocannya yak...
7,Itulah ceramah yg sebenarnya.
8,Besok bugil aja selawat
9,"Bedanya Gus Miftah dengan FPI / HTI bagaikan surga dan neraka. Di surga penuh kasih, di neraka penuh penghancuran."


## Read data from database

In [28]:
dataset

,comment
0,"WKWKWK, GAK SEKALIAN PAK, SELAWAT KE DPR... KAN BANYAK KORUPTOR DISANA"
1,Mantab.. ini br dakwah yg sejati.. kl d tempat umum mah dh biasa . Ini luar biasa.�
2,Salut buat Gus Miftah
3,yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo \/ sunan kali jaga...smua ada cara demi kebaikan... asal jng ajk dg cara berjinah .... lakukan dg porsi dan sikon yg tepat
4,Ada adabnya sholawat. Lebih baik ajak ke majelis ilmu..
5,"Setiap ulama punya jalan dan cara dakwah masing2.., jika kamu melihat kemunkaran lawan dengan tanganmu(kekuatanmu) jika tidak mampu lawan dengan mulutmu jika tidak mampu juga dengan hatimu itulah selemah lemahnya iman.., itu sudah menjadi kewajiban para ulama, kalo yg komen masih ada yg nyinyirin ulama berarti otaknya sudah disisipin politik.."
6,Umpanin nocannya yak...
7,Itulah ceramah yg sebenarnya.
8,Besok bugil aja selawat
9,"Bedanya Gus Miftah dengan FPI / HTI bagaikan surga dan neraka. Di surga penuh kasih, di neraka penuh penghancuran."


In [29]:
#removing symbol
def strip_symbol(text):
    return re.sub(r'[^\w\s]',' ',text)

#removing integer
def strip_integer(text):
    return re.sub(r'[\d+]', ' ', text)

#removing regex and tokenizing
def regex_tokenizer(text):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(text)

def make_it_lowercase(text):
    return text.lower()

In [30]:
dataset['comment'] = dataset['comment'].apply(lambda text:strip_symbol(text))
dataset['comment'] = dataset['comment'].apply(lambda text:strip_integer(text))
dataset['comment'] = dataset['comment'].apply(lambda text:make_it_lowercase(text))
dataset['comment'] = dataset['comment'].apply(lambda text:regex_tokenizer(text))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [31]:
dataset

,comment
0,"[wkwkwk, gak, sekalian, pak, selawat, ke, dpr, kan, banyak, koruptor, disana]"
1,"[mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]"
2,"[salut, buat, gus, miftah]"
3,"[yg, nyinyir, gak, pernah, lihat, dan, baca, atw, mengetahui, sejarah, dakwah, walisongo, sunan, kali, jaga, smua, ada, cara, demi, kebaikan, asal, jng, ajk, dg, cara, berjinah, lakukan, dg, porsi, dan, sikon, yg, tepat]"
4,"[ada, adabnya, sholawat, lebih, baik, ajak, ke, majelis, ilmu]"
5,"[setiap, ulama, punya, jalan, dan, cara, dakwah, masing, jika, kamu, melihat, kemunkaran, lawan, dengan, tanganmu, kekuatanmu, jika, tidak, mampu, lawan, dengan, mulutmu, jika, tidak, mampu, juga, dengan, hatimu, itulah, selemah, lemahnya, iman, itu, sudah, menjadi, kewajiban, para, ulama, kalo, yg, komen, masih, ada, yg, nyinyirin, ulama, berarti, otaknya, sudah, disisipin, politik]"
6,"[umpanin, nocannya, yak]"
7,"[itulah, ceramah, yg, sebenarnya]"
8,"[besok, bugil, aja, selawat]"
9,"[bedanya, gus, miftah, dengan, fpi, hti, bagaikan, surga, dan, neraka, di, surga, penuh, kasih, di, neraka, penuh, penghancuran]"


In [32]:
df=dataset[:500]
df['comment']

0      [wkwkwk, gak, sekalian, pak, selawat, ke, dpr, kan, banyak, koruptor, disana]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
1      [mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]                                                                                                                                                                                                                                                                                                                                                          

In [33]:
# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# factory = StemmerFactory()
# stemmer = factory.create_stemmer()

# df['stem_comment'] = df['comment'].apply(lambda x: [stemmer.stem(y) for y in x])

In [34]:
# df['stem_comment']

In [35]:
# df['stem_comment'].to_csv('stemmed_data.csv', index=False)
# dff = pd.read_csv('stemmed_data.csv', names =['stem_comment'])
# dff

In [36]:
singkatan_kamus_alay = pd.read_csv("colloquial-indonesian-lexicon.csv")
singkatan_kamus_alay = singkatan_kamus_alay[singkatan_kamus_alay.columns[0:2]]
# singkatan_kamus_alay

In [37]:
singkatan_dataset = pd.read_csv("kumpulan_singkatan_dan_kata_dasar.csv", names=['singkatan', 'kata asli'])
# singkatan_dataset

In [38]:
dataset['comment']

0         [wkwkwk, gak, sekalian, pak, selawat, ke, dpr, kan, banyak, koruptor, disana]                                                                                                                                                                                                                                                                                                                                                                                                          
1         [mantab, ini, br, dakwah, yg, sejati, kl, d, tempat, umum, mah, dh, biasa, ini, luar, biasa]                                                                                                                                                                                                                                                                                                                                                                                           
2         [salut, buat, gus, miftah]

In [39]:
#import tqdm
#utk show progress
from tqdm import tqdm

In [40]:
for i in tqdm(range(len(df['comment']))):
    for j in range(len(df['comment'][i])):
        try:
            words = df['comment'][i][j]
            df['comment'][i][j] = list(singkatan_kamus_alay['formal'])[(list(singkatan_kamus_alay['slang']).
                                                                            index(df['comment'][i][j]))]
            ynormalized = df['comment'][i][j]
            print(df['comment'][i][j])
        except:
            pass
            print(df['comment'][i][j])

  0%|▎                                                                                 | 2/500 [00:00<00:28, 17.25it/s]

wkwkwk
enggak
sekalian
pak
selawat
ke
dpr
kan
banyak
koruptor
disana
mantab
ini
baru
dakwah
yang
sejati
kalau
ada
tempat
umum
mah
deh
biasa
ini
luar
biasa
salut
buat
gus
miftah
yang
nyinyir
enggak
pernah
lihat
dan
baca
atau
mengetahui
sejarah
dakwah
walisongo
sunan
kali
jaga
semua
ada
cara
demi
kebaikan
asal
jangan
ajk
dengan
cara
berzina
lakukan
dengan
porsi


  1%|▉                                                                                 | 6/500 [00:00<00:36, 13.66it/s]

dan
sikon
yang
tepat
ada
adabnya
sholawat
lebih
baik
ajak
ke
majelis
ilmu
setiap
ulama
punya
jalan
dan
cara
dakwah
masing
jika
kamu
melihat
kemunkaran
lawan
dengan
tanganmu
kekuatanmu
jika
tidak
mampu
lawan
dengan
mulutmu
jika
tidak
mampu
juga
dengan
hatimu
itulah
selemah
lemahnya
iman
itu
sudah
menjadi
kewajiban
para
ulama
kalo
yang
komen
masih
ada
yang
nyinyir
ulama
berarti
otaknya
sudah
disisipin
politik


  2%|█▌                                                                               | 10/500 [00:00<00:28, 16.93it/s]

...


enggak
ada
yang
mau
kasih
dan
ini
pelajaran
buat
sih
bpk
nya
lain
kali
kalo
mau
jual
tanah
itu
harus
di
pikirin
kedepannya
apakah
kalo
suatu
saat
di
bangun
ghe
mana
hasilnya
mengganggu
atau
tidak
kasihan
ya
yang
ngebangun
sudah
punya
imb
belum
salah
satu
syarat
imb
kalo
enggak
salah
ada
persetujuan
tetangga
kiri
kanan
depan
belakang
tega
sih
kalo
menurut
gue
harta

 99%|███████████████████████████████████████████████████████████████████████████████▎| 496/500 [00:30<00:00,  8.21it/s]


enggak
dibawa
mati
bossss
menunggu
karma
saja
tnggal
kasih
jalan
saja
selebar
motor
kan
deh
selesai
tanah
aing
kumaha
aing
beli
tanah
itu
sama
beli
jalan
jangan
lewat
tanah
orang
sebelum
rumah
itu
tertutup
semua
akses
jalannya
kan
sih
sertifikat
sudah
ada
gambar
denahnya
kalo
sudah
jelas
enggak
ada
akses
jalannya
mestinya
yang
di
prioritaskan
adalah
membebaskan
memneli
tanah
tetangga
dulu
untuk
akses
jalannnya
setelah
clear
barulah
mikir

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:30<00:00, 16.34it/s]


ngebangun
rumahnya
ya
allah
tega
banget
lapor
polisi
saja
itu
ada
uu
nya
enggak
bisa
seenaknya
begitu
bangun
rumah
menghalangi
rumah
orang
lain


In [41]:
for i in tqdm(range(len(df['comment']))):
    for j in range(len(df['comment'][i])):
        try:
            words = df['comment'][i][j]
            df['comment'][i][j] = list(singkatan_dataset['kata asli'])[(list(singkatan_dataset['singkatan']).
                                                                            index(df['comment'][i][j]))]
            ynormalized = df['comment'][i][j]
            print(df['comment'][i][j])
        except:
            pass
            print(df['comment'][i][j])

  6%|████▋                                                                           | 29/500 [00:00<00:01, 268.59it/s]

wkwk
enggak
sekalian
pak
selawat
ke
dapur
kan
banyak
koruptor
disana
mantab
ini
baru
dakwah
yang
sejati
kalau
ada
tempat
umum
mah
deh
biasa
ini
luar
biasa
salut
buat
gus
miftah
yang
nyinyir
enggak
pernah
lihat
dan
baca
atau
mengetahui
sejarah
dakwah
walisongo
sunan
kali
jaga
semua
ada
cara
demi
kebaikan
asal
jangan
ajak
dengan
cara
berzina
lakukan
dengan
porsi
dan
sikon
yang
tepat
ada
adabnya
sholawat
lebih
baik
ajak
ke
majelis
ilmu
setiap
ulama
punya
jalan
dan
cara
dakwah
masing
jika
kamu
melihat
kemunkaran
lawan
dengan
tanganmu
kekuatanmu
jika
tidak
mampu
lawan
dengan
mulutmu
jika
tidak
mampu
juga
dengan
hatimu
itulah
selemah
lemahnya
iman
itu
sudah
menjadi
kewajiban
para
ulama
kalo
yang
komen
masih
ada
yang
nyinyir
ulama
berarti
otaknya
sudah
disisipin
politik
umpanin
nocannya
ya
itulah
ceramah
yang
sebenarnya
besok
bugil
saja
selawat
bedanya
gus
miftah
dengan
fpi
hti
bagaikan
surga
dan
neraka
di
surga
penuh
kasih
di
neraka
penuh
penghancuran
kenapa
di
tempat
kerjanya
mending
diunda

 16%|████████████▋                                                                   | 79/500 [00:00<00:01, 248.73it/s]


yang
lebih
baik
lebih
baik
gus
kamu
mengajak
para
wanita
itu
menutupi
auratnya
lokalisasi
harapan
sadar
setelah
selawat
selesai
acara
selanjutnya
bagus
lanjutkan
bagus
itu
biar
sadar
islam
tidak
mengajari
kaumnya
buka
aurat
bagus
lanjut
banser
anshor
mana
suaranya
ormas
liberal
fan
itu
ketika
simbah
yang
bicara
selalu
dibully
karena
kata
katanya
terkesan
emosional
nantang
nantang
padahal
ketika
tantangan
bisa
di
buktikan
oleh
yang
di
tantang
terus
diam
tidak
ksatria
ngakui
dan
sering
menjelek
jelekkan
orang
lain
inilah
yang
jadi
penyebab
sering
di
bully
dan
orang
lain
jadi
enggak
simpati
mungkin
dia
dengan
berkata
seperti
itu
merasa
hebat
dia
tidak
sadar
saat
nyapres
pun
rontok
saat
tahun
pun
keok
artinya
rakyat
enggak
percaya
seolah
bicara
dulu
urusan
belakang
seperti
janji
jalan
kaki
jogja
jakarta
terbukti
enggak
kan
ini
mungkin
hal
sepele
bagi
dia
tapi
itu
tertanam
di
ingatan
orang
hingga
berkesimpulan
kalo
hanya
bisa
mengomong
tok
nglakoni
ora
iso
karena
mendukung
jokowi
jadi
tida

 31%|████████████████████████▏                                                      | 153/500 [00:00<00:01, 285.23it/s]

...


jalan
saran
saya
bila
mentok
bapak
tempuh
jalur
hukum
perdata
saya
yakin
pasti
menang
pentingnya
etika
hidup
bertetangga
harus
saling
bertegur
sapa
dan
membangun
hubungan
baik
dengan
orang
lain
hak
memag
sudah
haknya
tetapi
apakah
enggak
sadar
hidup
ini
hanya
sementara
dan
harta
cuma
titipan
tetangga
masa
begitu
kejam
tetangga
ini
tidak
berkerpimanusiaan
naon
wae
masalahna
nu
nutupna
kos
bagong
teu
bisa
ngalieuk
kapan
tangkurakna
ngaliwat
nincak
jagat
batur
dasar
bangpak
tulak
bantingkeun
gaya
developer
nih
orang
bangun
tanpa
di
beri
jalan
keluar
enggak
yakin
punyaimb
mudah
dan
pak
eko
purnomo
dan
keluarga
mendapat
ganti
yang
lebih
baik
amin
sabar
ya
pak
eko
sudah
enggak
punya
rasa
kasihan
terhadap
sesama
itu
yang
baru
bangun
sudah
ada
tawaran
dari
pemilik
rumah
yang
terblokade
juta
tapi
tetap
saja
enggak
mau
mengasih
toleran
kejamnya
mau
hidup
sendiri
tuh
orang
nanti
kalo
mati
kubur
sendiri
sebelum
mati
bikin
liang
terlebih
dulu
kali
ya
kok
bisa
keluar
ijin
ngebangun
jadikan
kandang


 94%|██████████████████████████████████████████████████████████████████████████▎    | 470/500 [00:02<00:00, 140.50it/s]


setiap
bangunan
harusnya
mempunyai
imb
atau
kemungkinan
pembangunan
rumah
tidak
punya
imb
hal
seperti
ini
tetangga
bisa
menggugat
ke
dinas
tata
kota
kalau
tidak
salah
karena
mebangun
rumah
tidak
hanya
karena
punya
tanah
saja
bisa
dibangun
semaunya
harus
mempertimbangkan
kepentingan
umum
terutama
tetangga
sekitar
kasihan
tanah
itu
yang
akan
melapang
dan
menyempit
ketika
kita
mati
kelak
semoga
keluarga
tersebut
senantiasa
diberi
kesabaran
dan
ketabahan
harta
hanya
titipan
semua
milik
tuhan
viralin
saja
imbnya
enggak
beres
tuh
bisa
dibongkar
yaudah
jadi
tetangganya
saja
tetangganya
busuk
itu
mah
tetangganya
pada
sakit
jiwa
sadis
tetangganya
pada
enggak
mau
mati
kali
setahu
saya
ada
di
kuh
perdata
namanya
hak
servituut
coba
yang
ahli
hukum
bagaiman
itu
penjelasannya
terlepas
dari
p
eko
yang
tidak
bagus
dalam
bertetangga
harusnya
tetangga
yang
bangun
tembok
gank
punya
nurani
biar
otaknya
kepakek
dia
bisa
dituntut
kok
bisa
ngebangun
tanpa
imb
bikin
susah
orang
itu
namanya
sekalian
laporin
p

100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:02<00:00, 176.60it/s]


rt
saya
di
ajarkan
orang
tua
tanah
itu
punya
fungsi
ekonomi
dan
fungsi
sosial
pernah
ngalami
kasus
yang
sama
dengan
tetangga
dia
bangun
rumah
pakai
seluruh
tanahnya
sampai
enggak
punya
halaman
kiri
kanan
depan
belakang
tanah
depan
rumahnya
saya
beli
tapi
ingat
nasihat
orang tua
ya
saya
kasih
untuk
tetangga
itu
jalan
masuk
pas
motor
dan
halaman
rumah
dia
itu
ditanah
saya
bicarakan
dengan
aparat
terkait
kang
insya
allah
ada
solusinya
ini
sebuah
ujian
sabar
kang
terus
berusaha
mencari
solusi
yakin
allah
punya
cara
sendiri
untuk
memberi
solusinya
amin
dari
zaman
dahulu
sampai
kiamat
nanti
sih
kayak
yang
dzolim
akan
selalu
ada
dimuka
bumi
ini
jahat
banget
ih
setega
itu
keterlaluan
dan
kudu
viralkan
kubur
saja
kita
selalu
mohon
dilapangkan
ini
di
dunia
tega
nya
menyempitkan
orang
lain
taroin
uler
saja
disitu
biar
jadi
sarang
uler
kan
nanti
tuh
uler
cari
makan
ke
rumah
tetangga
kalo
ditanya
uler
dari
mana
bilang
saja
enggak
tau
kan
itu
rumah
kosong
hehe
cadas
kan
ide
saya
betul
setuju
buat
p

In [42]:
#dari sini keliatan yg udah diproses cuma dataset ke-500, 501 blm diperbaiki
dataset[0:505]

,comment
0,"[wkwk, enggak, sekalian, pak, selawat, ke, dapur, kan, banyak, koruptor, disana]"
1,"[mantab, ini, baru, dakwah, yang, sejati, kalau, ada, tempat, umum, mah, deh, biasa, ini, luar, biasa]"
2,"[salut, buat, gus, miftah]"
3,"[yang, nyinyir, enggak, pernah, lihat, dan, baca, atau, mengetahui, sejarah, dakwah, walisongo, sunan, kali, jaga, semua, ada, cara, demi, kebaikan, asal, jangan, ajak, dengan, cara, berzina, lakukan, dengan, porsi, dan, sikon, yang, tepat]"
4,"[ada, adabnya, sholawat, lebih, baik, ajak, ke, majelis, ilmu]"
5,"[setiap, ulama, punya, jalan, dan, cara, dakwah, masing, jika, kamu, melihat, kemunkaran, lawan, dengan, tanganmu, kekuatanmu, jika, tidak, mampu, lawan, dengan, mulutmu, jika, tidak, mampu, juga, dengan, hatimu, itulah, selemah, lemahnya, iman, itu, sudah, menjadi, kewajiban, para, ulama, kalo, yang, komen, masih, ada, yang, nyinyir, ulama, berarti, otaknya, sudah, disisipin, politik]"
6,"[umpanin, nocannya, ya]"
7,"[itulah, ceramah, yang, sebenarnya]"
8,"[besok, bugil, saja, selawat]"
9,"[bedanya, gus, miftah, dengan, fpi, hti, bagaikan, surga, dan, neraka, di, surga, penuh, kasih, di, neraka, penuh, penghancuran]"


In [43]:
#simpan hasil si df['stem_comment'] ke csv
df['comment'].to_csv('data_bersih_tanpa_stemming.csv', index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [44]:
dffx=pd.read_csv('data_bersih_tanpa_stemming.csv', names=['hasil_perbaikan'])
dffx

,hasil_perbaikan
0,"['wkwk', 'enggak', 'sekalian', 'pak', 'selawat', 'ke', 'dapur', 'kan', 'banyak', 'koruptor', 'disana']"
1,"['mantab', 'ini', 'baru', 'dakwah', 'yang', 'sejati', 'kalau', 'ada', 'tempat', 'umum', 'mah', 'deh', 'biasa', 'ini', 'luar', 'biasa']"
2,"['salut', 'buat', 'gus', 'miftah']"
3,"['yang', 'nyinyir', 'enggak', 'pernah', 'lihat', 'dan', 'baca', 'atau', 'mengetahui', 'sejarah', 'dakwah', 'walisongo', 'sunan', 'kali', 'jaga', 'semua', 'ada', 'cara', 'demi', 'kebaikan', 'asal', 'jangan', 'ajak', 'dengan', 'cara', 'berzina', 'lakukan', 'dengan', 'porsi', 'dan', 'sikon', 'yang', 'tepat']"
4,"['ada', 'adabnya', 'sholawat', 'lebih', 'baik', 'ajak', 'ke', 'majelis', 'ilmu']"
5,"['setiap', 'ulama', 'punya', 'jalan', 'dan', 'cara', 'dakwah', 'masing', 'jika', 'kamu', 'melihat', 'kemunkaran', 'lawan', 'dengan', 'tanganmu', 'kekuatanmu', 'jika', 'tidak', 'mampu', 'lawan', 'dengan', 'mulutmu', 'jika', 'tidak', 'mampu', 'juga', 'dengan', 'hatimu', 'itulah', 'selemah', 'lemahnya', 'iman', 'itu', 'sudah', 'menjadi', 'kewajiban', 'para', 'ulama', 'kalo', 'yang', 'komen', 'masih', 'ada', 'yang', 'nyinyir', 'ulama', 'berarti', 'otaknya', 'sudah', 'disisipin', 'politik']"
6,"['umpanin', 'nocannya', 'ya']"
7,"['itulah', 'ceramah', 'yang', 'sebenarnya']"
8,"['besok', 'bugil', 'saja', 'selawat']"
9,"['bedanya', 'gus', 'miftah', 'dengan', 'fpi', 'hti', 'bagaikan', 'surga', 'dan', 'neraka', 'di', 'surga', 'penuh', 'kasih', 'di', 'neraka', 'penuh', 'penghancuran']"
